# IPywidgets: unraveling the magic behind @interact

## Jeroen Demeyer (Ghent University), ICMS Edinburgh 2017

**Note**: some of the features in this notebook are only available with IPywidgets 6 (currently still in development), although most should work with older versions.

In [1]:
from ipywidgets import *

### Basic `@interact` usage

In [2]:
@interact
def f(x=["one", "two", "three"], y=(0,100)):
    print("x = {0}".format(x))
    print("y = {0}".format(y))

x = one
y = 50


The `@interact` decorator is a simple frontend for the `interactive` class, which can be used directly:

In [3]:
def f(x, y):
    print("x + y = {0}".format(x + y))

In [4]:
interactive(f, x=(0,300,30), y=(0,100))

x + y = 200


You can also directly pass a widget object instead of an *abbreviation* (for example, the `(0,300,30)` above)

In [5]:
interactive(f, x=IntSlider(value=12), y=IntSlider(value=34))

x + y = 46


### Widgets

We can play with the widgets themselves (as opposed to SageNB, where widgets only make sense for interactive functions):

In [6]:
s = IntSlider(min=20, max=50, value=42)
s

The two sliders displayed below are actually the same as above. Note that they are automatically synchronized:

In [7]:
s

In [8]:
s

We can get or set the value from Python code:

In [9]:
s.value

42

In [10]:
s.value = 23

### Traitlets

The magic which implements this synchronization between the sliders comes from the `traitlets` package:

In [11]:
from traitlets import HasTraits, Int, link

In [12]:
# MyClass instances have an attribute "num" which must be an integer
class MyClass(HasTraits):
    num = Int()

In [13]:
# The "num" keyword is handled automatically in __init__
o = MyClass(num=20)
o.num

20

Traitlets does type checking:

In [14]:
o.num = "Hello World"

TraitError: The 'num' trait of a MyClass instance must be an int, but a value of 'Hello World' <type 'str'> was specified.

We can use `num` as an attribute in the usual way (provided that we assign it an integer):

In [15]:
o.num = 32
o.num

32

We now link the `value` attribute of the slider `s` to the `num` attribute of our object `o`.
This link will ensure that those attributes are always the same:
whenever one is changed, the other will be changed to the same value.

In [16]:
s = IntSlider(min=20, max=50, value=42)
s

In [17]:
link( (s, "value"), (o, "num") )

Change the slider value above and then execute the cell below...

In [18]:
o.num

42

We can also define a callback function which will be called whenever the attribute is changed:

In [19]:
def callback(*args):  # traitlets passes some args, but we don't care
    print("o.num = {0}".format(o.num))

In [20]:
o.observe(callback)

In [21]:
o.num = 20

o.num = 20


In [22]:
s.value = 50

o.num = 50


Changing the slider will change `o.num`:

In [23]:
s

### Creating an interact by hand

We now have all ingredients to implement `@interact` by hand.

We create a vertical box containing 2 sliders and an Output widget. Note the `description` keyword, which gives the sliders a label:

In [24]:
s1 = IntSlider(description="x")
s2 = IntSlider(description="y")
out = Output()

In [25]:
w = VBox([s1, s2, out])
w

We define a callback and make sure it is called whenever `s1` or  `s2` change:

In [26]:
def callback(*args):
    # The statement "with out" ensures that the output goes
    # to the Output widget (which needs to be cleared first)
    with out:
        out.clear_output()
        print("x + y = {0}".format(s1.value + s2.value))

In [27]:
s1.observe(callback)
s2.observe(callback)

This is our interact:

In [28]:
w

### The `HTML` widget

The `HTML` widget is useful to display static text or to display some output.
This is a small variation of our interact which uses the `HTML` widget:

In [29]:
t = HTML("<h3>Compute the sum</h3>")
sx = IntSlider(description="x")
sy = IntSlider(description="y")
ssize = IntSlider(min=1, value=10, max=50, description="font size")
h = HTML()
w2 = VBox([t, sx, sy, ssize, h])

fmt = '''<span style="font-size:{0}pt"><i>x</i> + <i>y</i> = <b>{1}</b></span>'''
def callback2(*args):
    h.value = fmt.format(ssize.value, sx.value + sy.value)

sx.observe(callback2)
sy.observe(callback2)
ssize.observe(callback2)
callback2()

w2

In [30]:
from time import sleep
for _ in range(500):
    v = sy.value
    if v == sy.max:
        sy.value = sy.min
    else:
        sy.value = v + 1
    sleep(0.05)

### References

* IPywidgets documentation: https://ipywidgets.readthedocs.io/en/latest/index.html
* List of widgets: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html